In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
url = "https://failbondi.fail/?date="
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}
months_max_days = { "01": 31, "02": 28, "03": 31, "04": 30, "05": 31, "06": 30, "07": 31, "08": 31, "09": 30, "10": 31, "11": 30, "12": 31 }
month_days = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11","12","13","14","15","16","17","18","19","20", "21", "22", "23", "24", "25", "26", "27", "28", "29", "30", "31"]

In [ ]:
def get_html_from_url(url, headers):
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        # Creamos un objeto BeautifulSoup para poder navegar por el HTML.
        soup = BeautifulSoup(response.content, "html.parser")
        return soup
    return None

# print(main_content)

In [ ]:
def scraping_vuelos(html, fecha):
    # 1. Extraer encabezados
    headers = [th.text.strip() for th in html.find('thead').find_all('th')]

    # 2. Extraer filas
    rows = []
    table_body = html.find('tbody')
    for tr in table_body.find_all('tr'):
        cells = [td.text.strip() for td in tr.find_all('td')]
        rows.append(cells)

    # 3. Crear DataFrame para manipular fácilmente
    df = pd.DataFrame(rows, columns=headers)
    df['fecha'] = fecha

    return df



In [ ]:
lista_dfs = []
import time
import random
def save_report_by_month(year_month, max_days):
    for i in range(max_days):
        date = year_month + month_days[i]
        url_link = url + date
        main_content = get_html_from_url(url_link, headers)
        # Ejecutar la función
        try:
            df_iteracion = scraping_vuelos(main_content, date)
            lista_dfs.append(df_iteracion)
        except Exception as e:
            print(f"Error en fecha {date}: {e}")


    df_month = pd.concat(lista_dfs, ignore_index=True)

    # Guardar el resultado final
    df_month.to_csv(f"todos_los_vuelos_{year_month}.csv", index=False, sep=';')

    print(f"Filas obtenidas: {len(df_month)}")
    time.sleep(random.uniform(2, 5))

In [ ]:
year = "2025"
from datetime import datetime

for month, max_days in months_max_days.items():
    year_month = year + "-" + month
    inicio_peticion = time.time()
    save_report_by_month(year_month, max_days)

    # --- Log de control ---
    fin_peticion = time.time()
    duracion = fin_peticion - inicio_peticion
    hora_actual = datetime.now().strftime('%H:%M:%S')

    print(f"[{hora_actual}] Finalizado: {year_month} | Tiempo: {duracion:.2f}s")

[2026-01-02 19:11:29] Fecha 2025-01 procesado con éxito.
Error en fecha 2025-0228: 'NoneType' object has no attribute 'find'
[2026-01-02 19:12:13] Fecha 2025-02 procesado con éxito.
[2026-01-02 19:12:52] Fecha 2025-03 procesado con éxito.
[2026-01-02 19:13:32] Fecha 2025-04 procesado con éxito.
[2026-01-02 19:14:05] Fecha 2025-05 procesado con éxito.
[2026-01-02 19:14:42] Fecha 2025-06 procesado con éxito.
[2026-01-02 19:15:21] Fecha 2025-07 procesado con éxito.
[2026-01-02 19:16:00] Fecha 2025-08 procesado con éxito.
[2026-01-02 19:16:46] Fecha 2025-09 procesado con éxito.
[2026-01-02 19:17:21] Fecha 2025-10 procesado con éxito.
[2026-01-02 19:17:59] Fecha 2025-11 procesado con éxito.
[2026-01-02 19:18:39] Fecha 2025-12 procesado con éxito.


In [ ]:
!pip install xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 4.7 MB/s eta 0:00:00


In [ ]:
import os
carpeta_csv = '/content'  # Carpeta donde están tus archivos

os.listdir(carpeta_csv)

['.config',
 'todos_los_vuelos_2025-03.csv',
 'todos_los_vuelos_2025-05.csv',
 'todos_los_vuelos_2025-02.csv',
 'todos_los_vuelos_2025-06.csv',
 'todos_los_vuelos_2025-11.csv',
 'todos_los_vuelos_2025-04.csv',
 'todos_los_vuelos_2025-07.csv',
 'todos_los_vuelos_2025-10.csv',
 'todos_los_vuelos_2025-12.csv',
 'todos_los_vuelos_2025-09.csv',
 'todos_los_vuelos_2025-08.csv',
 '.ipynb_checkpoints',
 'todos_los_vuelos_2025-01.csv',
 'sample_data']

In [ ]:
import pandas as pd
import os

# 1. Configuración
carpeta_csv = '/content'  # Carpeta donde están tus archivos
archivo_excel_final = 'Reporte_Anual_Vuelos.xlsx'

# 2. Crear el objeto escritor de Excel
with pd.ExcelWriter(archivo_excel_final, engine='xlsxwriter') as writer:
    # Listamos todos los archivos en la carpeta que terminen en .csv o .txt
    archivos = [f for f in os.listdir(carpeta_csv) if f.endswith('.csv') or f.endswith('.txt')]

    for archivo in sorted(archivos):
        # Leer el archivo (ajusta el sep='|' si usaste ese delimitador)
        path_completo = os.path.join(carpeta_csv, archivo)
        df = pd.read_csv(path_completo, sep='|')

        # Limpiamos el nombre del archivo para que sea el nombre de la pestaña
        # (Excel no permite nombres de pestañas de más de 31 caracteres)
        nombre_pestaña = archivo.replace('.csv', '').replace('.txt', '')[:31]

        # Escribir el DataFrame en una pestaña específica
        df.to_excel(writer, sheet_name=nombre_pestaña, index=False)

        print(f"Añadida pestaña: {nombre_pestaña}")

file_size = os.path.getsize(archivo_excel_final)

print(f"\n¡Éxito! El archivo '{archivo_excel_final}' ha sido creado. Tamaño: {file_size}")

Añadida pestaña: todos_los_vuelos_2025-01
Añadida pestaña: todos_los_vuelos_2025-02
Añadida pestaña: todos_los_vuelos_2025-03
Añadida pestaña: todos_los_vuelos_2025-04
Añadida pestaña: todos_los_vuelos_2025-05
Añadida pestaña: todos_los_vuelos_2025-06
Añadida pestaña: todos_los_vuelos_2025-07
Añadida pestaña: todos_los_vuelos_2025-08
Añadida pestaña: todos_los_vuelos_2025-09
Añadida pestaña: todos_los_vuelos_2025-10
Añadida pestaña: todos_los_vuelos_2025-11
Añadida pestaña: todos_los_vuelos_2025-12

¡Éxito! El archivo 'Reporte_Anual_Vuelos.xlsx' ha sido creado.
